In [ ]:
from langchain_community.llms import Ollama
from langchain.agents import tool
from langchain.agents import tool, initialize_agent, AgentType, Tool
import sys
import os


sys.path.append('../../bluesky')  # Adjust the path as necessary


# Now you can import bluesky modules

sys.path.append(os.path.abspath('../'))
import os
import streamlit as st

import bluesky

from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

from langchain.agents.format_scratchpad.openai_tools import (

    format_to_openai_tool_messages,

)


from langchain_community.tools.tavily_search import TavilySearchResults

from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import ChatOpenAI

from openai import OpenAI

from io import StringIO

from contextlib import contextmanager
import time

from dotenv import load_dotenv, find_dotenv

import chromadb.utils.embedding_functions as embedding_functions

import chromadb

from bluesky.network.client import Client

from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper

from prompts.prompts import conflict_prompt


from langchain.output_parsers import ResponseSchema, StructuredOutputParser

from langchain_experimental.utilities import PythonREPL

from langchain.agents import Tool

os.environ["LANGCHAIN_PROJECT"] = "Communication-Errors"

In [ ]:
# Load the .env file
load_dotenv(find_dotenv())

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-large"
)

vectordb_path = os.getcwd() + '/../skills-library/vectordb'
#vectordb_path = 'C:/Users/justa/OneDrive/Desktop/Developer/LLM-Enhanced-ATM/llm/skills-library/vectordb'
chroma_client = chromadb.PersistentClient(path=vectordb_path)
# Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection = chroma_client.get_or_create_collection(
    name="test1", embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})
collection.peek()

In [ ]:
# capture output information from the bluesky client and return it as a string
@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def update_until_complete(client):
    complete_output = ""
    empty_output_count = 0  # Track consecutive empty outputs

    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()

        # Check if the current output is empty
        if not new_output.strip():
            empty_output_count += 1  # Increment counter for empty outputs
        else:
            empty_output_count = 0  # Reset counter if output is not empty
            complete_output += new_output  # Add non-empty output to complete output

        # If there are two consecutive empty outputs, break the loop
        if empty_output_count >= 3:
            break

    # It's assumed you want to keep the last update outside the loop
    client.update()

    return complete_output

In [ ]:
# connect to client
client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()
client.update()

In [ ]:
with open('../prompts/basecmds.txt', 'r') as file:
    base_cmds = file.read()

Creating tools for agent

In [ ]:

# get all aircraft info
# get conflict information

@tool
def GetAllAircraftInfo(command: str = 'GETACIDS'):
    """Get each aircraft information at current time: position, heading (deg), track (deg), altitude, V/S (vertical speed), calibrated, true and ground speed and mach number. Input is 'GETACIDS'.
    
    Parameters:
    - command: str (default 'GETACIDS')
    
    Example usage:
    - GetAllAircraftInfo('GETACIDS')
    
    Returns:
    - str: all aircraft information
    """
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    print(f'LLM input:{command}')
    
    client.send_event(b'STACK', command)
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output

@tool
def GetConflictInfo(commad: str = 'SHOWTCPA'):
    """Use this tool to identify and get vital information on aircraft pairs in conflict. It gives you Time to Closest Point of Approach (TCPA), Quadrantal Direction (QDR), separation distance, Closest Point of Approach distance (DCPA), and Time of Loss of Separation (tLOS).
    
    Parameters:
    - command: str (default 'SHOWTCPA')
    
    Example usage:
    - GetConflictInfo('SHOWTCPA')
    
    Returns:
    - str: conflict information between aircraft pairs
    """
    client.send_event(b'STACK', 'SHOWTCPA')
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output


@tool
def ContinueMonitoring(time: str = '5'):
    """Monitor for conflicts between aircraft pairs for a specified time. 
    Parameters:
    - time (str): The time in seconds to monitor for conflicts. Default is 5 seconds.
    
    Example usage:
    - ContinueMonitoring('5')
    
    Returns:
    - str: The conflict information between aircraft pairs throughout the monitoring period.
    """
    for i in range(int(time)):
        client.send_event(b'STACK', 'SHOWTCPA')
        time.sleep(1)
        sim_output += str(i) + ' sec: \n' + update_until_complete(client) + '\n'
    return sim_output


@tool
def SendCommand(command: str):
    """
    Sends a command with optional arguments to the simulator and returns the output. 
    You can only send 1 command at a time.
    
    Parameters:
    - command (str): The command to send to the simulator. Can only be a single command, with no AND or OR operators.
    
    Example usage:
    - SendCommand('COMMAND_NAME ARG1 ARG2 ARG3 ...) # this command requires arguments
    - SendCommand('COMMAND_NAME') # this command does not require arguments
    
    Returns:
    str: The output from the simulator.
    """
    # Convert the command and its arguments into a string to be sent
    #command_with_args = ' '.join([command] + [str(arg) for arg in args])
    # Send the command to the simulator
    # client.update()  # Uncomment this if you need to update the client state before sending the command
    print(command)
    # replace " or ' in the command string with nothing
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    client.send_event(b'STACK', command)  
    # wait 1 second
    time.sleep(1)
    # Wait for and retrieve the output from the simulator
    sim_output = update_until_complete(client)
    if sim_output == '':
        return 'Command executed successfully.'
    if 'Unknown command' in sim_output:
        return sim_output + '\n' + 'Please use a tool QueryDatabase to search for the correct command.'
    return sim_output



@tool
def QueryDatabase(input: str):
    """If you want to send command to a simulator please first search which command you should use. For example if you want to create an aircraft, search for 'how do I create an aircraft'.
    Parameters:
    - input: str (the query to search for)
    Returns:
    - list: the top 10 results from the database
    """

    query_results = collection.query(
        query_texts=[input],
        n_results=5
    )

    
    return query_results['documents'][0]


tools = [GetAllAircraftInfo, GetConflictInfo,
         SendCommand, QueryDatabase]

In [ ]:
input = "check if there are any conflicts between aircraft and if there are resolve them by changing altitude or heading. Your Goal is to have no conflicts between aircraft."

In [ ]:
# Get the prompt to use - you can modify this!
from langchain import hub
react_prompt = hub.pull("hwchase17/react-chat")
react_prompt.pretty_print()

In [ ]:

# react_prompt.template = """
# Answer the following questions as best you can. You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, specify exactly one of the following tools [{tool_names}]. No additional text or comments permitted.
# Action Input: the input argument to the action.
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question




# Begin!

# Question: {input}
# Thought:{agent_scratchpad}
# """
# react_prompt.pretty_print()



In [ ]:
xml_prompt = hub.pull("hwchase17/xml-agent-convo")
xml_prompt.pretty_print()

 Agent

In [ ]:
# Get the prompt to use - you can modify this!
openai_function_prompt = hub.pull("hwchase17/openai-functions-agent")
openai_function_prompt.messages[0].prompt.template = """
You are an elite air traffic assistant that is monitoring the communication between the pilots and the air traffic controller. Your task is to detect call sign errors, ambiguities, misinterpretation and if you detect any, provide a clarification to the pilots or the air traffic controller. Provide clarification message in this format: Sender: LLM, Receiver: <Here you decide who should receive the message, can be multiple receivers>, message: <Your Message>. After providing the message try to fix the situation by using tools. Use these tools:
GetAllAircraftInfo - This will output all the information about current aircraft in the airspace
GetConflictInfo - This will output the information about any conflicts between aircraft
SendCommand - This will send a command to the simulator. You can use this to control the aircraft
QueryDatabase - This will search the database for the correct command to send to the simulator. Use this if you are unsure of the command to send to the simulator.

Be a proactive air traffic assistant, use tools as much as possible to get the information you need to provide the correct clarification.

Examples of call sign errors:
- similar call signs: "DAL123" and "DAL132". Command is for "DAL123" but "DAL132" responded or both responded.
- Incomplete call signs: command sent to "Delta123" instead of "DAL123 Heavy"
- Readback call sign error: command sent to "XYZ123" but "XYZ132" readback the command as "XYZ123" or "XYZ123" readback the command as "XYZ132"
- Misheard call sign: command sent to "ABC123" but "ABC123" heard it as "ABC132" so "ABC123" never responded.
- Wrong prefix: command sent to "American123" but was meant for "Delta123"
- wrong suffix: command sent to "KLM123" but was meant for "KLM123Heavy"
- Misheard Call Sign: command sent to ABC123 but ABC123 misheard as another call sign and it didn't respond.


Notes:
there could be commands to aircraft that don't even exits and was meant to be for another aircraft creating a call sign error. for example "AAL123" instead of "DAL123", so you always need to check which aircraft currently exist in airspace with GetAllAircraftInfo tool.

If the command is clear, just say "Command is clear".
"""


openai_function_prompt.pretty_print()

In [ ]:
import time
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, create_json_chat_agent, create_xml_agent, create_openai_functions_agent
from langchain_openai import OpenAI, ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_anthropic import AnthropicLLM, ChatAnthropic
from langchain_google_genai import GoogleGenerativeAI
from langchain.memory import ConversationSummaryBufferMemory

#     model="gemini-pro", google_api_key=os.getenv("GEMINI_API_KEY"))
memory = ConversationSummaryBufferMemory(
    llm=OpenAI(), max_token_limit=1000, memory_key="chat_history", return_messages=True)
# model_name = "llama2:70b"
# llm = Ollama(base_url="https://ollama.junzis.com", model=model_name)
llm = ChatOpenAI(model='gpt-4-turbo-2024-04-09')
def invoke_until_success(llm):
    while True:
        try:
            # Attempt to invoke the method
            llm.invoke('hi')
            print("Invocation successful.")
            break  # Exit the loop if invocation was successful
        except Exception as e:
            print("Invocation failed, trying again...")
            time.sleep(1)  # Wait for a short period before retrying

# Example usage
# invoke_until_success(llm)
#invoke_until_success(llm)

react_agent = create_react_agent(llm, tools, react_prompt)

xml_agent = create_xml_agent(llm, tools, xml_prompt)

openai_function_agent = create_openai_functions_agent(llm, tools, openai_function_prompt)

# Create an agent executor by passing in the agent and tools
react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True, memory=memory)

xml_agent_executor = AgentExecutor(
    agent=xml_agent, tools=tools, verbose=True, handle_parsing_errors=True, memory=memory)

openai_function_executor = AgentExecutor(
    agent=openai_function_agent, tools=tools, verbose=True, handle_parsing_errors=True, memory=memory)
#json_agent_executor = AgentExecutor(agent=json_agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
client.send_event(b'STACK', 'IC simple/communication/call_sign_errors/readback.scn')

In [ ]:
update_until_complete(client)

In [ ]:
# import time
# # get time in hh:mm:sec
# def get_time():
#     return time.strftime("%H:%M:%S", time.localtime())

In [ ]:
# current_time = get_time()
# input = f"TIME: {current_time} SOURCE: BA023 MESSAGE: VR342, roger, continuing taxi, holding short of Runway 27."

In [ ]:
# import csv


# def read_csv_to_list(filename):
#     # List to store the formatted data
#     formatted_data = []

#     # Open the CSV file
#     with open(filename, newline='', encoding='utf-8') as csvfile:
#         # Create a CSV reader specifying the delimiter as '|'
#         reader = csv.DictReader(csvfile, delimiter='|')

#         # Iterate over each row in the CSV file
#         for row in reader:
#             # Format each row as specified and add it to the list
#             formatted_row = f"time: {row['time']}, source: {row['source']}, message: {row['message']}"
#             formatted_data.append(formatted_row)

#     return formatted_data


# formatted_data = read_csv_to_list('../data/conversations/tenerife.csv')

In [ ]:
def read_file_to_list(file_path):
    """
    Reads a text file and returns a list of lines.

    Args:
    file_path (str): The path to the text file to be read.

    Returns:
    list: A list where each element is a line from the file.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines in the file
        # Strip whitespace from each line
        lines = [line.strip() for line in lines]
    return lines


# Example usage:
lines = read_file_to_list(
    "../data/conversations/call_sign_errors/readback.txt")
print(lines)

In [ ]:
for i in range(len(lines)):
    print(lines[i])
    out = openai_function_executor.invoke({"input": lines[i]})

In [ ]:
import langsmith

from datetime import datetime, timedelta


# langsmith knows which client by api from .env file
langsmith_client = langsmith.Client()

# Load the chat history from Langsmith
chat_history_df = langsmith_client.get_test_results(
    project_name="Communication-Errors")
# Filter chat history to only include questions where the model did not know the answer by searching 'sorry' in the answer
chat_history_df = chat_history_df[['input.input', 'outputs.output']]
# remove rows with nan
chat_history_df = chat_history_df.dropna()
# reset index
chat_history_df = chat_history_df.reset_index(drop=True)


# only keep rows 1 to 169
chat_history_df = chat_history_df.iloc[0:96]

# reorder from bottom to top
chat_history_df = chat_history_df.iloc[::-1]

# save to csv, seperator is |



chat_history_df.to_csv('../data/conversation_with_llm/tenerife.csv', index=False, sep='|')